### Data extraction

In [1]:
from utils import fetch_log_returns
training_dataset = fetch_log_returns(start = '1995-01-01', end ='1995-12-31')

[*********************100%***********************]  5 of 5 completed


In [2]:
training_dataset.values

array([[-8.03420318e-04, -1.03968302e-02, -6.34465166e-03,
        -3.45697194e-03, -1.95645036e-02],
       [ 7.38303962e-04,  3.28007496e-03,  1.07261458e-02,
        -4.93917085e-03,  0.00000000e+00],
       [ 3.25539389e-04, -2.33000766e-03, -3.00612927e-03,
        -3.82611528e-03, -1.43458320e-02],
       ...,
       [ 3.74407369e-04, -1.62287613e-04,  4.93541396e-03,
         5.36316418e-03, -2.68870072e-02],
       [-6.67454827e-04, -2.87200507e-03,  8.16115201e-05,
        -6.95147773e-03,  1.73196363e-02],
       [ 2.94296798e-03, -5.82632554e-03,  3.42115394e-03,
        -2.50644600e-04,  8.74144700e-03]])

### Define the problem instance parameters

In [3]:
from model_params import LAMBDA_1, LAMBDA_2, LAMBDA_3, NLAYERS, NSHOTS, NUM_ASSETS, SIGMA_TARGET, TWO_QUBIT_GATES, K, N

### Create the Ansatz for VQE (No angle embedding)

In [4]:
from ansatz import build_hardware_efficient_ansatz

# Example usage
print(build_hardware_efficient_ansatz(num_qubits=5, num_layers=3).draw())


q0: ─U2─U2─o───────U1─U2─o───────U1─U2─o───────U1─M─
q1: ─U2─U2─X─o─────U1─U2─X─o─────U1─U2─X─o─────U1─M─
q2: ─U2─U2───X─o───U1─U2───X─o───U1─U2───X─o───U1─M─
q3: ─U2─U2─────X─o─U1─U2─────X─o─U1─U2─────X─o─U1─M─
q4: ─U2─U2───────X─U1─U2───────X─U1─U2───────X─U1─M─


### Building the Portfolio Optimization Hamiltonian and running VQE


In [5]:
import numpy as np
from qibo.optimizers import optimize
from cost_function import compute_total_energy

# Optimize starting from a random guess for the variational parameters
initial_params = np.random.uniform(0, 2*np.pi, 2*N*NLAYERS + N)
# data = np.random.normal(0, 1, size=2**nqubits)

# perform optimization
best, params, extra = optimize(compute_total_energy, initial_params, args=(training_dataset))

# set final solution to circuit instance
print('Ground state:', best)

print('Optimal parameters', params)

print('Optimization process info', extra)

[Qibo 0.2.8|INFO|2024-10-03 10:02:53]: Using numpy backend on /CPU:0


Ground state: 0.050492103202664666
Optimal parameters [6.08010533 1.22981948 1.19123188 1.05053993 2.49523301 5.51950142
 1.74155263 4.61501606 4.92551146 0.08562469 1.8042936  2.00541475
 5.1204532  1.36050054 1.97727759 3.15993379 5.25427823 1.78947582
 5.8213211  3.83515348 1.94540998 4.76259525 1.45985886 3.13175756
 3.27778741]
Optimization process info  message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 0.050492103202664666
       x: [ 6.080e+00  1.230e+00 ...  3.132e+00  3.278e+00]
     nit: 1
   direc: [[ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
           [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
           ...
           [ 0.000e+00  0.000e+00 ...  1.000e+00  0.000e+00]
           [ 0.000e+00  0.000e+00 ...  0.000e+00  1.000e+00]]
    nfev: 76
